[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/01_fastai_pets.ipynb)

# Vorbereitungen in Colab

Wenn du ein Notebook in Google Colab öffnest, musst du den "Laufzeit-Typen" den Instanz auf "GPU" ändern. Damit bekommst die Instanz eine GPU zugewiesen und das Training ist deutlich schneller. 

Menü "Laufzeit" -> "Laufzeittyp ändern"

<div>
<img src="imgs/01_runtime_type.png" width="600">
</div>

Im Pop-Up den Hardwarebeschleuniger auf "GPU" ändern und "Speichern".

<div>
<img src="imgs/02_runtime_type_gpu.png" width="400">
</div>

Die Instanz wird jetzt neu geladen und du musst alle Zellen noch einmal ausführen!

# Pakete installieren und importieren

Python-Pakete von [fast.ai](https://www.fast.ai) installieren und importieren.

In [ ]:
!pip install -Uq fastai

In [ ]:
from fastai.vision.all import *

**Wichtig!** Wenn die folgende Zelle "False" ausgibt, ist keine GPU vorhanden -> Bitte prüfe die o. g. Schritte noch einmal und starte ggf. das Notebook neu. Bei "True" hast du eine GPU-Instanz und kannst weiter machen.

In [ ]:
torch.cuda.is_available()

# Daten laden

Fast.ai liefert verschiedene Standard-Datensets mit, die häufig für DeepLearning-Benchmarks verwendet werden. Das gebräuchlichste ist das PETS-Dataset. Es besteht aus 37 verschiedenen Hunde- und Katzenrassen und umfasst insgesamt ca. 7400 Bilder.

Datensatz herunterladen und den Pfad als `path`-Variable speichern. Anschlißend die Bilderpfade als `files`-Liste laden.

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
(path/'images').ls()

In [ ]:
files = get_image_files(path/"images")
len(files)

Jupyter Notebooks unterstützt die Anzeige von Bildern. Dazu  müssen wir mit `PILImage.create` ein Bild-Objekt (Variable `img`) erstellen und zeigen es an. Um ein anderes Bild anzuzeigen, kannst du die Varible `img_number` ändern und die Zelle erneut ausführen.

In [ ]:
img_number = 7
img = PILImage.create(files[img_number])
img

# Cats-vs-Dogs

Als einfachstes Modell trainieren wir einen Cats-vs-Dogs Classifier. D.h. das Modell wird trainiert um zwischen Hunden und Katzen zu unterscheiden. Dazu machen wir uns zur Nutze, dass in diesem Datensatz, die Katzenbilder mit einen Großbuchstaben beginnen (Egyptian_Mau_167.jpg) und die Hundebilder mit Kleinbuchstaben (shiba_inu_136.jpg).

In [ ]:
def label_func(f): return 'Cat' if f[0].isupper() else 'Dog'

Damit wir unser Modell mit den Bildern "füttern" können, müssen wir einen Dataloader erstellen. Der Dataloader ist dafür zuständig, die Bilder zu laden und mit den passenden Labels (bzw. Klassen - in diesem Beispiel Cat / Dog zu versehen). Außerdem müssen alle Bilder auf dieselbe Größe zugeschnitten werden (hier 224 x 224 Pixel).

In [ ]:
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

In [ ]:
dls.show_batch()

Deep Learning Modell verarbeiten die Daten in Batches. D. h. die Bilder werden nicht einzeln, sondern in "Batches" von 64  Bildern (die Standard Batch-Size) zusammengefasst und verarbeitet.

Mit `dls.one_batch()` laden wir einen Batch und zeigen uns die `shape` an. 

**(Batch-Size, Farb-Kanäle (rot, grün, blau), Höhe, Breite)**

In [ ]:
batch = dls.one_batch()
batch[0].shape

Der Dataloader hat im Hintergrund die Bilder in einen `Tensor` umgewandelt. Man kann vereinfacht sagen, dass alle Input-Daten die ein Deep Learning Modell verarbeit soll, in einen Tensor umgewandelt werden muss. Ein Tensor ist ein mehrdimensionales Array, das 32-Bit Float Zahlen enthält.

Unsere Bilder sind `TensorImage`s:

In [ ]:
batch[0]

Unsere Kategorien (Cat / Dog) wurden auch durch den Dataloader in Tensoren (`TensorCategory`) umgewandelt:

In [ ]:
batch[1]

# Modell trainieren

Wir trainieren ein Resnet34-Modell - genauer gesagt nehmen wir ein "Pretrained"-Modell und Fine-tunen es. Wie gut unser Modell funktioniert, können wir an der Genauigkeit (accuracy) ablesen. Die accuracy zeigt uns, welchen Prozentsatz der Bilder korrekt kalssifiziert wurden.

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(5)

# Ergebnisse anzeigen

`learn.show_results()` zeigt Beispielbilder und deren Klassifizierung an (rot bedeutet, dass das Modell das Bild falsch klassifiziert hat).

In [ ]:
learn.show_results()

Mit `learn.predict()` können wir ein einzelnes Bild von unserem Modell klassifizieren lassen.

In [ ]:
img_to_predict = files[1337]
learn.predict(img_to_predict)

Liegt das Modell richtig? Die folgende Zelle zeigt das Bild an:

In [ ]:
PILImage.create(img_to_predict)